In [ ]:
import pandas as pd
from pyproj import Proj, transform
import rasterio
import rioxarray
import xarray as xr

# Assuming your DataFrame is named df and has columns 'lat_local' and 'lon_local' for the Swiss coordinates
# Example:
# df = pd.DataFrame({'station_name': ['Station1', 'Station2'], 'lat_local': [600000, 605000], 'lon_local': [200000, 205000]})

# Define the projection for Swiss coordinates (CH1903/LV03) and WGS84
proj_swiss = Proj(init='epsg:21781')  # CH1903/LV03
proj_wgs84 = Proj(init='epsg:4326')  # WGS84

# Convert coordinates
df_coords['lon_wgs84'], df_coords['lat_wgs84'] = transform(p1=proj_swiss, 
                                                           p2=proj_wgs84, 
                                                           x=df_coords['X'].values, 
                                                           y=df_coords['Y'].values)

# Load the SRTM raster data with rioxarray
srtm_file = '../../data/idaweb/switzerland.tif'
srtm_data = rioxarray.open_rasterio(srtm_file)

# Function to extract nearest elevation value for given coordinates
def get_elevation(lat, lon, srtm_data):
    # Use sel method to select nearest point, method='nearest' ensures closest point is chosen
    elevation = srtm_data.sel(x=lon, y=lat, method='nearest').values
    return elevation.item()  # Convert numpy array to scalar

# Apply the function to each row in the DataFrame to create a new 'elevation' column
df_coords['Z'] = df_coords.apply(lambda row: get_elevation(row['lat_wgs84'], row['lon_wgs84'], srtm_data), axis=1)

In [6]:
merged_df = df.merge(df_coords[['X','Y','Z']],how='inner',on=['X','Y'])

In [8]:
merged_df.rename(columns={'Z_y':'SRTM', 'Z_x':'Z'}, inplace=True)

In [10]:
merged_df.to_pickle(PATH, protocol=4)